<a href="https://colab.research.google.com/github/alonlavian/AlmaZohar/blob/master/FaceCrop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trying to enlarge my data set using Google vision API

For the following to work you'll have to setup:
*   service account credentials - https://cloud.google.com/docs/authentication/production#obtaining_and_providing_service_account_credentials_manually
*   google billing  - https://support.google.com/googleapi/answer/6158867?hl=en



###  Install google cloud vision

In [0]:
!pip install --upgrade google-cloud-vision



### Mount Drive

In [0]:
import os 
from google.colab import drive
drive.mount('/content/drive/')

### Add vision application credentilas to env.

In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/ML/Misc/vision_service_account.json"

### Cropping code

In [0]:
# https://cloud.google.com/vision/docs/face-tutorial
# https://cloud.google.com/vision/docs/crop-hints

import os
from google.cloud import vision
from google.cloud.vision import types
from PIL import Image

def detect_face(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of Face objects with information about the picture.
    """
    client = vision.ImageAnnotatorClient()

    content = face_file.read()
    image = types.Image(content=content)

    return client.face_detection(image=image, max_results=max_results).face_annotations
  


def crop_faces(image_to_crop, cropped_photos_dir):
    """Crop faces, then saves to output_filename.

    Args:
      image_to_crop: a file containing the image with the faces.
      cropped_photos_dir: the name of the directory 
    """
    
    with open(image_to_crop, 'rb') as image:
        faces = detect_face(image, 8)
        print('Found {} face{}'.format(
            len(faces), '' if len(faces) == 1 else 's'))
        
        
    im = Image.open(image_to_crop)

    for idx,face in enumerate(faces):
   
      vects = face.fd_bounding_poly.vertices
      im2 = im.crop([vects[0].x, vects[0].y,
                  vects[2].x - 1, vects[2].y - 1])
    
      original_image_name = os.path.basename(image_to_crop)
      cropped_image_name = os.fsdecode(os.path.splitext(original_image_name)[0]) + "_face_" +str(idx) +".jpg"
      cropped_image_path = os.path.join(os.fsencode(cropped_photos_dir),os.fsencode(cropped_image_name))
      im2.save(cropped_image_path, 'JPEG')
      print('Saved new image to :' ,cropped_image_path)
    
    
    

### Cropping one image

In [9]:
path = os.fsencode('/content/drive/My Drive/Google Photos/2019/04/IMG_20190402_075133.jpg') 
filename = os.fsencode('IMG_20190402_075133.jpg')
cropped_photos_dir = os.fsencode('/content/')
crop_faces(path, cropped_photos_dir)

Found 2 faces
Saved new image to : b'/content/IMG_20190402_075133_face_0.jpg'
Saved new image to : b'/content/IMG_20190402_075133_face_1.jpg'


### Cropping 100 random images from every month of 2017

In [0]:

    
    
import random


original_photos_dir = os.fsencode('/content/drive/My Drive/Google Photos/2017')
cropped_photos_dir  = os.fsencode('/content/drive/My Drive/ML/AlmaZoharCrop/CropMix')
if not os.path.exists(cropped_photos_dir):
    os.makedirs(cropped_photos_dir)
    
    
for path, dirs, files in os.walk(original_photos_dir):
  if files:
    try:
      rand_files = (random.sample(files,100))
    except:
      continue    
    for file in rand_files:
      filename = os.fsdecode(file)
      if filename.endswith(".jpg"):
        image_to_crop = os.path.join(path,file)
        print("Cropping ", filename)
        crop_faces(image_to_crop,cropped_photos_dir)            